# Prepare files used in AEP steady state study.
### 1. Get the zone numbers in each county in Texas.

In [ ]:
"""This section finds the zone numbers in each county in texas."""
import numpy as np
import pandas as pd

# Read the excel file, user needs to change the path accordingly!!!!!!
df_county = pd.read_excel("data/ERCOT_24SSWGU1_Planning_Data_Dictionary_11072024.xlsx",sheet_name=2,usecols="B,S")      # planning data dictionary
df_zone = pd.read_excel("data/24SSWG_2025_SUM1_Final_10142024.xlsx")   # data exported from a raw file

# get bus numbers in each county
bus_in_county = df_county.groupby('PLANNING BUS COUNTY')['SSWG BUS NUMBER'].apply(list).to_dict()
bus_in_zone = df_zone.groupby('Zone Num')['Bus Number'].apply(list).to_dict()

# get zone numbers in each county
county_list = list(bus_in_county.keys())
county_list = sorted(list(set([county.title() for county in county_list])))
zone_list = [1,2,3,4,5,6,7,11,12,13,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,40,41,42,44,45,46,47,48,49,50,51,53,54,55,57,60,61,62,63,67,70,71,72,102,103,104,105,106,107,108,109,110,113,114,115,118,119,120,121,122,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,141,142,143,145,146,147,148,149,150,151,152,153,154,155,156,157,158,160,161,162,163,164,165,166,167,168,169,170,171,172,177,178,180,181,182,183,184,185,186,187,199,200,220,221,222,224,225,226,227,229,230,233,234,238,240,260,261,295,301,302,303,304,305,308,310,317,318,319,320,350,351,393,394,424,428,432,434,438,442,444,446,456,458,460,462,466,472,474,477,478,479,500,502,504,505,506,508,510,511,512,514,516,517,519,522,525,526,527,528,531,534,537,540,542,543,546,549,551,552,553,554,555,558,561,564,566,567,570,571,572,574,575,576,577,578,579,580,581,582,583,584,585,587,588,589,590,591,592,593,594,610,612,615,620,625,630,635,636,640,645,651,659,669,670,671,672,673,674,675,676,691,692,695,709,712,790,794,800,825,829,870,872,874,875,876,878,880,882,884,886,890,891,900,901,902,903,904,906,907,911,918,920,930,932,935,937,939,941,942,948,951,952,960,970,971,972,973,974,975,976,977,979,980,984,985,986,987,988,989,991,992,993,994,1000,1001,1003,1004,1006,1007,1009,1011,1012,1013,1014,1015,1016,1017,1019,1020,1023,1024,1026,1027,1029,1033,1034,1036,1037,1039,1041,1043,1044,1046,1047,1050,1051,1052,1053,1054,1055,1057,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,1070,1071,1072,1073,1074,1075,1078,1079,1081,1082,1083,1084,1091,1092,1094,1095,1097,1098,1099,1100,1102,1106,1108,1109,1110,1111,1112,1113,1114,1122,1123,1126,1128,1129,1131,1133,1134,1136,1140,1141,1143,1146,1150,1152,1154,1156,1157,1160,1162,1163,1164,1166,1167,1171,1172,1175,1176,1177,1178,1179,1180,1181,1183,1184,1189,1190,1193,1194,1196,2006,2012,2028,2029,2030,2037,2040,2052,2075,2077,2081,2103,2107,2110,2119,2133,2136,2141,2144,2160,2162,2177,2183,2204,2206,2208,2213,2215,2220,2231,2234,2239,2252,2253,9000]

# initial the dictionary where the key is the county and the values are the zone numbers in that county
zone_in_county = {county.title():[] for county in county_list}

# check each bus in a county
for county,buses in bus_in_county.items():
    for bus_number in buses:
        mask = df_zone['Bus Number'].values == bus_number
        if not np.all(mask==False): # bus with a new zone number
            df_bus = df_zone.loc[mask]
            zone_number = int(df_bus.iloc[0]['Zone Num'])

            # add zone number to the list
            if zone_number not in zone_in_county[county.title()]:
                zone_in_county[county.title()].append(zone_number)
                zone_in_county[county.title()].sort()


### 2. Define a Class called "County", Texas as a dictionary of County class objects.

In [ ]:
"""This section defined a County class and initialize them."""
class County:
    """A class to model the counties in Texas state"""

    def __init__(self, name, neighbor, zone):
        """Initialize name, neighbor counties and zone numbers attributes."""
        'name is a string, neighbor is a list of strings'
        self.name = name
        self.neighbor = neighbor
        self.zone = zone
        self.neighbor_sort()
        self.zone.sort()

    def neighbor_sort(self):
        """Sort the neighbors based on the names."""
        sorted_neighbor = sorted(self.neighbor, key=lambda x: x.name)
        self.neighbor = sorted_neighbor

    def add_neighbors(self, counties):
        """Add neighbor counties."""
        for county in counties:
            if county in self.neighbor:
                print(f"{county.name} is already in the neighbor list of {self.name}.")
            else:
                self.neighbor.append(county)
        self.neighbor_sort()        

    def add_zones(self, numbers):
        """Add zone numbers in this county."""
        for number in numbers:
            if number in self.zone:
                print(f"{number} is already in the zone list of {self.name}.")
            else:
                self.zone.append(number)
        self.zone.sort()

    def remove_neighbors(self,counties):
        """Remove counties from neighbor list."""
        for county in counties:
            if county in self.neighbor:
                self.neighbor.remove(county)
            else:
                print(f"{county.name} is not in the neighbor list of {self.name}.")

    def remove_zones(self, numbers):
        """Remove zone numbers from this county."""
        for number in numbers:
            if number in self.zone:
                self.zone.remove(number)
            else:
                print(f"{number} is not in the zone list of {self.name}.")

    def print_neighbors(self):
        """Print the names of the neighbor counties."""
        county_name_list = []
        for county in self.neighbor:
            county_name_list.append(county.name)

        print(f"{self.name} has neighbor county: {county_name_list}.")

    def print_zones(self):
        """Print the zone numbers in this county."""
        print(f"{self.name} has zone number: {self.zone}.")

    def get_area_zones(self, neighbors):
        """Return a list of zone numbers which covers the county and its neighbors."""

        """The zone number list of Texas."""
        outer_area = [1,2,3,4,5,6,7,11,12,13,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,40,41,42,44,45,46,47,48,49,50,51,53,54,55,57,60,61,62,63,67,70,71,72,102,103,104,105,106,107,108,109,110,113,114,115,118,119,120,121,122,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,141,142,143,145,146,147,148,149,150,151,152,153,154,155,156,157,158,160,161,162,163,164,165,166,167,168,169,170,171,172,177,178,180,181,182,183,184,185,186,187,199,200,220,221,222,224,225,226,227,229,230,233,234,238,240,260,261,295,301,302,303,304,305,308,310,317,318,319,320,350,351,393,394,424,428,432,434,438,442,444,446,456,458,460,462,466,472,474,477,478,479,500,502,504,505,506,508,510,511,512,514,516,517,519,522,525,526,527,528,531,534,537,540,542,543,546,549,551,552,553,554,555,558,561,564,566,567,570,571,572,574,575,576,577,578,579,580,581,582,583,584,585,587,588,589,590,591,592,593,594,610,612,615,620,625,630,635,636,640,645,651,659,669,670,671,672,673,674,675,676,691,692,695,709,712,790,794,800,825,829,870,872,874,875,876,878,880,882,884,886,890,891,900,901,902,903,904,906,907,911,918,920,930,932,935,937,939,941,942,948,951,952,960,970,971,972,973,974,975,976,977,979,980,984,985,986,987,988,989,991,992,993,994,1000,1001,1003,1004,1006,1007,1009,1011,1012,1013,1014,1015,1016,1017,1019,1020,1023,1024,1026,1027,1029,1033,1034,1036,1037,1039,1041,1043,1044,1046,1047,1050,1051,1052,1053,1054,1055,1057,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,1070,1071,1072,1073,1074,1075,1078,1079,1081,1082,1083,1084,1091,1092,1094,1095,1097,1098,1099,1100,1102,1106,1108,1109,1110,1111,1112,1113,1114,1122,1123,1126,1128,1129,1131,1133,1134,1136,1140,1141,1143,1146,1150,1152,1154,1156,1157,1160,1162,1163,1164,1166,1167,1171,1172,1175,1176,1177,1178,1179,1180,1181,1183,1184,1189,1190,1193,1194,1196,2006,2012,2028,2029,2030,2037,2040,2052,2075,2077,2081,2103,2107,2110,2119,2133,2136,2141,2144,2160,2162,2177,2183,2204,2206,2208,2213,2215,2220,2231,2234,2239,2252,2253,9000]

        """Add zone in itself."""
        study_area = self.zone
        for number in self.zone:
            outer_area.remove(number)
        
        """Add zone in neighbors."""
        for county in neighbors:
            if county in county_list:
                county_ob = texas[county.title()]
                for number in county_ob.zone:
                    if number not in study_area:
                        study_area.append(number)
                        outer_area.remove(number)
        
        study_area.sort()
        outer_area.sort()

        return study_area, outer_area

"""Define counties with the zone numbers."""
texas = {}
for county in county_list:
    texas[county] = County(county,[],zone_in_county[county])

### 3. Define function to get all study area counties.

In [ ]:
def get_adj(county):
    """Input the county name e.g. Anderson County"""

    import requests
    from bs4 import BeautifulSoup

    county = county.replace(" ","_")
    response = requests.get("https://en.wikipedia.org/wiki/"+county+",_Texas")
    soup = BeautifulSoup(response.text, "lxml")
    links = (
        soup
        .select_one("#Adjacent_counties,#Adjacent_counties_and_municipalities,#Adjacent_counties_and_parish,#Adjacent_counties_and_parishes")
        .parent
        .find_next_sibling(["div","ul"])
        .find_all("a", href=True)
    )
    adj = []
    for link in links:
        if 'Texas' in link["title"]:
            adj.append(link["title"][:-7])

    return sorted(adj)

def get_adj2(county):
    """Input the county name e.g. Anderson County"""

    adj_counties = get_adj(county)
    adj2 = get_adj(county)
    for adj in adj_counties:
        adj_adj_counties = get_adj(adj)
        for adj_adj in adj_adj_counties:
            if adj_adj not in adj2 and adj_adj != county:
                adj2.append(adj_adj)

    return sorted(adj2)

### 4. Create the subsystem file and monitor file for the input project.

In [ ]:
"""This section generates the subsystem file and monitor file for given county in Texas."""
"""Run the previous section first."""
import os

# User input all the informations of the project
name = input("Please enter the name of the project (Use & for \'and\'): ")
project_number = input("Please enter the project number (2XINRXXXX): ")
poi_county = input("Please enter the POI county:")
# counties_string = input("Please enter other counties in study area, seperate with comma: ")

# Test
# name = 'test'
# poi_county = 'Webb'
# counties_string = 'Maverick, Dimmit, La Salle, McMullen, Duval, Jim Hogg, Zapata,Kinney, Zavala, Frio, Atascosa, Live Oak, Jim Wells, Brooks, Starr'

# User needs to change this path accordingly!!!!
path = 'data/AEP Steady State/'+project_number+' ' +name+'/'
# Check whether the specified path exists or not
isExist = os.path.exists(path)
if not isExist:

   # Create a new directory because it does not exist
   os.makedirs(path)

counties_list = [county[:-7] for county in get_adj2(poi_county+' County')]
# counties_string.replace(", ", ",").split(',')

# Find the County Object matching name in the texas list
# county = next((cnty for cnty in texas if cnty.name == county_name), None)
county = texas[poi_county.title()]

# Get the zone numbers in the study area and outer area
study_area, outer_area = county.get_area_zones(counties_list)

"""Create the subsystem file."""
filename = path+name+'.sub'

with open(filename, 'w') as file_object:
    file_object.write("COM   **************************************************\n")
    file_object.write("COM   *           PROJECT-RELATED SUBSYSTEM            *\n")
    file_object.write("COM   *        CREATED BY ARTHUR LI - CF POWER         *\n")
    file_object.write("COM   **************************************************\n\n")

    file_object.write(f"Subsystem '{name}'"+"\n    BUS\nEnd\n")
    file_object.write(f"Subsystem 'Study_Area'"+"\n")
    # Add zones to the study area
    for number in study_area:
        file_object.write(f"    ZONE {number}\n")
    file_object.write("End\n\nSubsystem 'Outer_Area'"+"\n")
    # Add zones to the outer area
    for number in outer_area:
        file_object.write(f"    ZONE {number}\n")
    file_object.write("End")

"""Create the monitor file."""
filename = path+name+'.mon'

with open(filename, 'w') as file_object:
    file_object.write("COM   **************************************************\n")
    file_object.write("COM   *           PROJECT-RELATED SUBSYSTEM            *\n")
    file_object.write("COM   *        CREATED BY ARTHUR LI - CF POWER         *\n")
    file_object.write("COM   **************************************************\n\n")
    file_object.write("MONITOR BRANCHES IN SUBSYSTEM 'Study_Area'\n")
    file_object.write("MONITOR TIES FROM SUBSYSTEM  'Study_Area'\n")
    file_object.write("MONITOR VOLTAGE RANGE SUBSYSTEM 'Study_Area' 0.92 1.05 0.95 1.05 // N-1 N-0\n")
    file_object.write("MONITOR VOLTAGE DEVIATION SUBSYSTEM 'Study_Area' kvrange 100 765 0.08")

### 5. Create the combined contigency files for the input project in section 4.

In [ ]:
"""This section generates the contigency files for each cases based on year and season. Years for 3 seasons are needed."""
import os

# different year of contigency files given by ERCOT
sum_year_list = ['2024','2025','2026','2027','2028','2029','2030','2031']
min_year_list = ['2028']
spg_year_list = ['2025']

# User input to get the years (4-digits string)
sum_year = input("Please enter the year of your summer case: ")
# min_year = input("Please enter the year of your minimum case: ")
# spg_year = input("please enter the year of your spring case: ")
min_year = 2028
spg_year = 2025

season_list = ['SUM','MIN','SPG']
year_list = [sum_year, min_year, spg_year]
psse_year_list = [sum_year_list, min_year_list, spg_year_list]
i = 0

# User needs to change this original contigency file path accordingly!!!!!!!
con_path = "data/ERCOT_24SSWG_Contingencies/PSSE"
# This is the path where the combined contigency files should be saved.
path = 'data/'+project_number+' ' +name+'/con file/'

# Check whether the specified path exists or not
isExist = os.path.exists(path)
if not isExist:

   # Create a new directory because it does not exist
   os.makedirs(path)

# counties_list = counties_string.replace(", ", ",").split(',')

# Check if the user input year is available
for year in year_list:
    if year not in psse_year_list[i]:
        if i == 0:
            print(f"{year} summer contigency file is not available.")
        elif i == 1:
            print(f"{year} minimum contigency file is not available.")
        else:
            print(f"{year} spring contigency file is not available.")


    # Find the path of the contigency files
    if i == 1:
        season_path = con_path+"//24SSWG_"+year+"_"+season_list[i]
    else:
        season_path = con_path+"//24SSWG_"+year+"_"+season_list[i]+"1"

    dir_list = os.listdir(season_path)
    # Combine contigency files based on category
    if i == 2:
        con_type = {'N-1' : ['P1','P2.1','P7'],}
    else:
        con_type = {
            'N-1' : ['P1','P2.1','P7'],
            'P4&P5' : ['P4','P5'],
            'N-G&N-A' : ['P1.1','P1.3'],
                    }

    # Create contingency file for different types.
    for key, value in con_type.items():
        new_filename = path + f"{year[2:]}{season_list[i]}_{key}.con"
        for filename in dir_list:
            if any(x in filename for x in value):
                with open(season_path+"/"+filename) as file_object:
                    contents = file_object.read().rstrip()
                with open(new_filename,'a') as file_object:
                    file_object.write(contents.rstrip('END'))

    # Create an all-together contingency file.
    all_filename = path + f'{year[2:]}{season_list[i]}_All.con'
    for filename in dir_list:
        with open(season_path+"/"+filename) as file_object:
            contents = file_object.read()
        with open(all_filename,'a') as file_object:
            file_object.write(contents)
    i += 1